## Ćwiczenie 4 - Odtworzenie zaimplementowanej architektury sieci 
Maciej Dutkowski 260396

In [2]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [12]:
#warnings.filterwarnings('ignore')

In [3]:
heart_disease = fetch_ucirepo(id=45) 
  
x = heart_disease.data.features 
y = heart_disease.data.targets

In [4]:
categorical_features_labels = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal', 'ca']
numerical_features_labels = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

Usuwanie rekordów z brakującymi wartościami

In [5]:
indices_to_keep = x.dropna().index
x = x.loc[indices_to_keep].reset_index()
y = y.loc[indices_to_keep].reset_index()

Zamiana etykiet na zbiór {0,1}

In [6]:
y = y['num'].apply(lambda x: 0 if x == 0 else 1)

Przekształcenie cech kategorycznych na wartości binarne one-hot-encoding i normalizacja cech liczbowych

In [7]:
#One hot encoding
x = pd.get_dummies(x, columns = categorical_features_labels)
x = x.reset_index(drop=True)
boolean_columns = x.select_dtypes(include=bool).columns
x[boolean_columns] = x[boolean_columns].astype(int)

# Normalizacja
scaler = MinMaxScaler()
x[numerical_features_labels] = scaler.fit_transform(x[numerical_features_labels])

x = x.drop("index", axis=1)

In [8]:
display(x,y)

,age,trestbps,chol,thalach,oldpeak,sex_0,sex_1,cp_1,cp_2,cp_3,...,slope_1,slope_2,slope_3,thal_3.0,thal_6.0,thal_7.0,ca_0.0,ca_1.0,ca_2.0,ca_3.0
0,0.708333,0.481132,0.244292,0.603053,0.370968,0,1,1,0,0,...,0,0,1,0,1,0,1,0,0,0
1,0.791667,0.622642,0.365297,0.282443,0.241935,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
2,0.791667,0.245283,0.235160,0.442748,0.419355,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
3,0.166667,0.339623,0.283105,0.885496,0.564516,0,1,0,0,1,...,0,0,1,1,0,0,1,0,0,0
4,0.250000,0.339623,0.178082,0.770992,0.225806,1,0,0,1,0,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,0.583333,0.433962,0.262557,0.396947,0.032258,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
293,0.333333,0.150943,0.315068,0.465649,0.193548,0,1,1,0,0,...,0,1,0,0,0,1,1,0,0,0
294,0.812500,0.471698,0.152968,0.534351,0.548387,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
295,0.583333,0.339623,0.011416,0.335878,0.193548,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,0


0      0
1      1
2      1
3      0
4      0
      ..
292    1
293    1
294    1
295    1
296    1
Name: num, Length: 297, dtype: int64

Podział na zbiór testowy i treningowy

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4, random_state = 42)

### Konfiguracja sieci

Jako framework do głębokiego uczenia wybrałem biblotekę tensorflow. Pozwala skonfigurować bardzo wiele paraemtrów sieci i może wykorzystywać procesoru GPU do przeprowadzania obliczeń.

In [241]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import HeNormal, GlorotUniform
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

Jako pierwsza została zdefiniowana funkcja tworząca model sieci. Wymiary sieci, funkcje aktywacji jak i strategie inicjalizacji wag dla poszczególnych warstw wybrałem takie same jak dla wcześniejszego ćwiczenia dla najlepszej sieci jaką udało mi się uzyskać. Kształt sieci to 3 warstwy i na każdej po 2 neurony (oprócz ostatniej). Na warstwach ukrytych jako funkcja aktywacji jest ReLU i He inicjalizacja. Natomiast na wartstwie wyjściowej jest sigmoid i Xavier/Glorot inicjalizacja. Wymiary sieci, funkcje aktywacji jak i strategie inicjalizacji konfiguruje obiektem Sequential biblioteki tensorflow.

In [242]:
seed = 42
def create_model():    
    return Sequential([
        Dense(2, activation='relu', input_shape=(28,), kernel_initializer = HeNormal(seed=seed)),
        Dense(2, activation='relu', kernel_initializer = HeNormal(seed=seed)),
        Dense(1, activation='sigmoid', kernel_initializer = GlorotUniform(seed=seed))
    ])

Następnie tworze funkcję kompilującą model. Podejemy jej w parametrze wcześniej utworzony model oraz optymalizator. Ustawiamy w niej funkcję kosztu dla sieci (entropia krzyżowa z poprzedniego ćwiczenia) oraz metryki, które chcemy mierzyć (dokładność modelu).

In [243]:
def compile_model(model, optimizer):    
    model.compile(
        optimizer = optimizer,
        loss = 'binary_crossentropy',
        metrics = ['accuracy']
    )

Zostaje również zdefiniowana funkcja trenująca, która dla ustalonej liczby epok (50) oraz danego rozmiaru batcha trenuje model.

In [244]:
def train_model(model, batch_size):    
    model.fit(
        x_train, y_train,
        epochs = 50,
        batch_size = batch_size,
        verbose = 0
    )

Funkcja testująca dokładność modelu.

In [245]:
def test_model(model):    
    _, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    return test_accuracy


Następnie cały proces uczenia i testowania zostaje zamknięty w jednej funkcji, tak by można było łątwo testować go dla różnych hiperparametrów.

In [273]:
def process_model(optimizer, batch_size=10):
    model = create_model()
    compile_model(model=model, optimizer=optimizer)
    train_model(model=model, batch_size=batch_size)
    return test_model(model)

### Wyniki

#### W zależności od optymalizatora

In [274]:
accuracy_sgd = process_model(optimizer = SGD(learning_rate=0.01))
accuracy_adam = process_model(optimizer = Adam(learning_rate=0.01))
accuracy_rms = process_model(optimizer = RMSprop(learning_rate=0.01))

print("Accuracy of SGD: ", accuracy_sgd)
print("Accuracy of Adam: ", accuracy_adam)
print("Accuracy of RMSprop: ", accuracy_rms)

Accuracy of SGD:  0.756302535533905
Accuracy of Adam:  0.8235294222831726
Accuracy of RMSprop:  0.7899159789085388


Jak widać najlepsze wyniki dla wspołczynnika uczenia wynoszącego 0.01, rozmiarze batcha 10 i liczbie epok 50 odniósł optymalizator typu Adam (~82%), następnie RMSprop(~79%) oraz najgorzej SDG (~75%). Wynikać to może między innymi z tego, ze zarówno Adam jak i RMSprop automatycznie dostosowywują szybkość uczenia dla poszczególnych wag, dzięki czemu mogą być bardziej efektywne niż SDG, który wykorzystuje stałą szybkość uczenia. Dodatkowo Adam wykorzystuje pęd przy aktualizacji wag (uwzględniający wcześniejsze aktualizacje), który zmniejsza jego szansę na utknięciu w minimum lokalnym, co mogło wpłynąć na to że miał najlepszy wynik.

#### W zależności od rozmiaru batcha

In [282]:
accuracy_sgd_batch_lowest = process_model(optimizer = SGD(learning_rate=0.01), batch_size=3)
accuracy_sgd_batch_low = process_model(optimizer = SGD(learning_rate=0.01), batch_size=5)
accuracy_sgd_batch_medium = process_model(optimizer = SGD(learning_rate=0.01), batch_size=10)
accuracy_sgd_batch_high = process_model(optimizer = SGD(learning_rate=0.01), batch_size=15)

print("Batch size 3: ", accuracy_sgd_batch_lowest)
print("Batch size 5: ", accuracy_sgd_batch_low)
print("Batch size 10: ", accuracy_sgd_batch_medium)
print("Batch size 15: ", accuracy_sgd_batch_high)

Batch size 3:  0.8235294222831726
Batch size 5:  0.831932783126831
Batch size 10:  0.756302535533905
Batch size 15:  0.5042017102241516


Jak widać dla SGD najlepszy wynik został uzyzkany dla rozmiaru batcha 5. Duże batche pozwalają w bardziej dokładny sposób wyliczyć gradient (ponieważ jest uśredniony dla wielu próbek), ale również narażają model na utknięcie w minimum lokalnym.

#### W zależności od wartości współczynnika uczenia

In [276]:
accuracy_sgd_10 = process_model(optimizer = SGD(learning_rate=1))
accuracy_sgd_1 = process_model(optimizer = SGD(learning_rate=0.1))
accuracy_sgd_01 = process_model(optimizer = SGD(learning_rate=0.01))
accuracy_sgd_001 = process_model(optimizer = SGD(learning_rate=0.001))

print("SGD learning_rate 1.0: ", accuracy_sgd_10)
print("SGD learning_rate 0.1: ", accuracy_sgd_1)
print("SGD learning_rate 0.01: ", accuracy_sgd_01)
print("SGD learning_rate 0.001: ", accuracy_sgd_001)

SGD learning_rate 1.0:  0.5042017102241516
SGD learning_rate 0.1:  0.8067227005958557
SGD learning_rate 0.01:  0.756302535533905
SGD learning_rate 0.001:  0.5042017102241516


In [281]:
accuracy_adam_01 = process_model(optimizer = Adam(learning_rate=0.01))
accuracy_adam_001 = process_model(optimizer = Adam(learning_rate=0.001))
accuracy_adam_0001 = process_model(optimizer = Adam(learning_rate=0.0001))

print("Adam learning_rate 0.01: ", accuracy_adam_01)
print("Adam learning_rate 0.001: ", accuracy_adam_001)
print("Adam learning_rate 0.0001: ", accuracy_adam_0001)

Adam learning_rate 0.01:  0.8235294222831726
Adam learning_rate 0.001:  0.848739504814148
Adam learning_rate 0.0001:  0.5042017102241516


Jak widać dla SDG najlepszym współczynnikiem uczenia okazał się być 0.1 a dla Adama 0.001. Różnica może wynikać z tego, że Adam stosuje inny algorytm aktualizacji wag, który nie jest oparty wyłącznie na współczynniku uczenia i gradienicie ale również pędzie wyliczanym na podstawie wcześniejszych aktualizacji.